In [ ]:
# import pickle
# filename = 'data/cui_data_v3_with_trial_baseline_no_repetitaion_42776.pkl'

# with open(filename, 'rb') as handle:
#     b = pickle.load(handle)

# print(len(b))
# print(len(b[0]))

In [ ]:
# lines = b[0]
# print(lines[:2])

# print(len(lines))

# with open('data/baseline_bert_train_raw_v1.txt', 'w') as f:
#     for line in lines:
#         line = line.replace('[CLS] ', '').replace(' [SEP] ', ' ')
#         f.write(f"{line}\n")

In [1]:
!nvidia-smi

Tue Aug  9 12:44:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 59%   88C    P2   160W / 250W |   6580MiB / 11019MiB |     97%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 32%   

In [2]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"
from transformers import AutoModelForMaskedLM, AutoTokenizer

/home/hl2659/miniconda3/envs/hao_ner/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# t_model = "bert-base-uncased"
model_type = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
model = AutoModelForMaskedLM.from_pretrained(model_type)
tokenizer = AutoTokenizer.from_pretrained(model_type, use_fast=True)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# input_data_file = '../data/raw_data_v3_without_trial_repetitaion_60198.txt'
input_data_file = 'data/raw_data_v3_with_trial_repetitaion_155287.txt'

from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        file_path=input_data_file,
        block_size=256)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

/home/hl2659/miniconda3/envs/hao_ner/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="output_sensitive",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=12,
    save_steps=5000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

/home/hl2659/miniconda3/envs/hao_ner/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 133110
  Num Epochs = 2
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 22186


Step,Training Loss
500,1.147500
1000,1.133400
1500,1.106700
2000,1.100900
2500,1.107700
3000,1.099000
3500,1.103600
4000,1.077100
4500,1.054500
5000,1.049400


Saving model checkpoint to output_sensitive/checkpoint-5000
Configuration saved in output_sensitive/checkpoint-5000/config.json
Model weights saved in output_sensitive/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to output_sensitive/checkpoint-10000
Configuration saved in output_sensitive/checkpoint-10000/config.json
Model weights saved in output_sensitive/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to output_sensitive/checkpoint-15000
Configuration saved in output_sensitive/checkpoint-15000/config.json
Model weights saved in output_sensitive/checkpoint-15000/pytorch_model.bin
Deleting older checkpoint [output_sensitive/checkpoint-5000] due to args.save_total_limit
Saving model checkpoint to output_sensitive/checkpoint-20000
Configuration saved in output_sensitive/checkpoint-20000/config.json
Model weights saved in output_sensitive/checkpoint-20000/pytorch_model.bin
Deleting older checkpoint [output_sensitive/checkpoint-10000] due to args.save_total_limit




TrainOutput(global_step=22186, training_loss=0.9605989764812005, metrics={'train_runtime': 5300.0074, 'train_samples_per_second': 50.23, 'train_steps_per_second': 4.186, 'total_flos': 3.0445666663707e+16, 'train_loss': 0.9605989764812005, 'epoch': 2.0})

In [6]:
trainer.save_model("output_sensitive/")

Saving model checkpoint to output_sensitive/
Configuration saved in output_sensitive/config.json
Model weights saved in output_sensitive/pytorch_model.bin


In [ ]:

# Retrieve pretrained model
# model = AutoModelForMaskedLM.from_pretrained("output3/")

In [ ]:
# t_model = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(t_model, use_fast=True)
# import pickle
# filename = 'data/cui_voc_pubmed_abtracts_v3.pickle'

# with open(filename, 'rb') as handle:
#     b = pickle.load(handle)

# print(len(b))
# blist = list(b)
# blist = [x.lower() for x in blist]
# blist.sort(key=str.lower)
# blist[:10]
# # Let's increase the vocabulary of Bert model and tokenizer
# new_tokens = blist
# num_added_toks = tokenizer.add_tokens(new_tokens)
# print('We have added', num_added_toks, 'tokens')
# # Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
# model.resize_token_embeddings(len(tokenizer))
# print(len(tokenizer))

In [ ]:
# !pip install datasets git+https://github.com/huggingface/transformers/


In [ ]:
# import os
# # os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
# from transformers import AutoModelForMaskedLM, AutoTokenizer

In [ ]:
# model = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)
# model = AutoModelForMaskedLM.from_pretrained(model)

In [ ]:
# import pickle
# filename = 'data/cui_voc_pubmed_abtracts_v3.pickle'

# with open(filename, 'rb') as handle:
#     b = pickle.load(handle)

# print(len(b))

In [ ]:
# blist = list(b)
# blist = [x.lower() for x in blist]
# blist.sort(key=str.lower)
# blist[:10]

In [ ]:
# print(len(tokenizer))

In [ ]:
# # Let's increase the vocabulary of Bert model and tokenizer
# new_tokens = blist
# num_added_toks = tokenizer.add_tokens(new_tokens)
# print('We have added', num_added_toks, 'tokens')
# # Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# 'c0043541' in tokenizer.vocab

In [ ]:
# text = "Here is the sentence I want embeddings for c0043541 ."

# marked_text = "[CLS] " + text + " [SEP]"

# # Tokenize our sentence with the BERT tokenizer.
# tokenized_text = tokenizer.tokenize(marked_text)

# # Print out the tokens.
# print(tokenized_text)

In [ ]:
# from transformers import LineByLineTextDataset
# dataset = LineByLineTextDataset(
#         tokenizer=tokenizer,
#         file_path="data/exist_cui_def_bert_train.txt",
#         block_size=256)

# from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=True, mlm_probability=0.15
# )

In [ ]:
# from transformers import Trainer, TrainingArguments


# training_args = TrainingArguments(
#     output_dir="output3",
#     overwrite_output_dir=True,
#     num_train_epochs=10,
#     per_device_train_batch_size=8,
#     save_steps=100,
#     save_total_limit=2
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=dataset
# )

# trainer.train()

In [ ]:
# trainer.save_model("output3/")

In [ ]:
# from transformers import BertForSequenceClassification

# model = BertForSequenceClassification.from_pretrained("output2", num_labels=2)



In [7]:
from transformers import BertModel, BertTokenizer

# Retrieve SciBERT.
model = BertModel.from_pretrained("output_sensitive/", output_hidden_states=True)

loading configuration file output_sensitive/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file output_sensitive/pytorch_model.bin
Some weights of the model checkpoint at output_sensitive/ were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions

In [8]:
import numpy as np

def get_word_indeces(tokenizer, text, word):
    '''
    Determines the index or indeces of the tokens corresponding to `word`
    within `text`. `word` can consist of multiple words, e.g., "cell biology".
    
    Determining the indeces is tricky because words can be broken into multiple
    tokens. I've solved this with a rather roundabout approach--I replace `word`
    with the correct number of `[MASK]` tokens, and then find these in the 
    tokenized result. 
    '''
    # Tokenize the 'word'--it may be broken into multiple tokens or subwords.
    word_tokens = tokenizer.tokenize(word)

    # Create a sequence of `[MASK]` tokens to put in place of `word`.
    masks_str = ' '.join(['[MASK]']*len(word_tokens))

    # Replace the word with mask tokens.
    text_masked = text.replace(word, masks_str)

    # `encode` performs multiple functions:
    #   1. Tokenizes the text
    #   2. Maps the tokens to their IDs
    #   3. Adds the special [CLS] and [SEP] tokens.
    input_ids = tokenizer.encode(text_masked)

    # Use numpy's `where` function to find all indeces of the [MASK] token.
    mask_token_indeces = np.where(np.array(input_ids) == tokenizer.mask_token_id)[0]

    return mask_token_indeces

In [15]:
import torch
def get_embedding(b_model, b_tokenizer, text, word=''):
    '''
    Uses the provided model and tokenizer to produce an embedding for the
    provided `text`, and a "contextualized" embedding for `word`, if provided.
    '''

    # If a word is provided, figure out which tokens correspond to it.
    if not word == '':
        word_indeces = get_word_indeces(b_tokenizer, text, word)

    # Encode the text, adding the (required!) special tokens, and converting to
    # PyTorch tensors.
    encoded_dict = b_tokenizer.encode_plus(
                        text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        return_tensors = 'pt',     # Return pytorch tensors.
                )

    input_ids = encoded_dict['input_ids']
    
    b_model.eval()

    # Run the text through the model and get the hidden states.
    bert_outputs = b_model(input_ids)
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():

        outputs = b_model(input_ids)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]

    # `hidden_states` has shape [13 x 1 x <sentence length> x 768]

    # Select the embeddings from the second to last layer.
    # `token_vecs` is a tensor with shape [<sent length> x 768]
    token_vecs = hidden_states[-2][0]
    

    # Calculate the average of all token vectors.
#     sentence_embedding = torch.mean(token_vecs, dim=0)
    #     sentence_embedding = torch.sum(token_vecs, dim=0)
    last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
    # cast layers to a tuple and concatenate over the last dimension
    cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.size())

    # take the mean of the concatenated vector over the token dimension
    cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
#     print(cat_sentence_embedding)
#     print(cat_sentence_embedding.size())
    sentence_embedding = cat_sentence_embedding

    # Convert to numpy array.
    sentence_embedding = sentence_embedding.detach().numpy()

    # If `word` was provided, compute an embedding for those tokens.
    if not word == '':
        # Take the average of the embeddings for the tokens in `word`.
        word_embedding = torch.mean(token_vecs[word_indeces], dim=0)

        # Convert to numpy array.
        word_embedding = word_embedding.detach().numpy()
    
        return (sentence_embedding, word_embedding)
    else:
        return sentence_embedding

In [10]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# # text = "hydrogels are hydrophilic polymer networks which may absorb from 10–20% (an arbitrary lower limit) up to thousands of times their dry weight in water."
# # word = 'hydrogels'

# # text = 'C0007222 (CVDs) are the leading cause of death globally, taking an estimated 17.9 million lives each year. '
# # word = 'C0007222'

# text = 'C0007222'
# word = 'C0007222'

# tokenized_text = tokenizer.tokenize(text)
# print(tokenized_text)

# # Get the embedding for the sentence, as well as an embedding for 'hydrogels'.
# (sen_emb, word_emb) = get_embedding(model, tokenizer, text, word)

# print('Embedding sizes:')
# print(sen_emb.shape)
# print(word_emb.shape)
# # Embedding sizes:
# # (768,)
# # (768,)
# # Here’s the code for calculating cosine similarity. We’ll test it by
# #  comparing the word embedding with the sentence embedding–not a very interesting comparison, but a good sanity check.

# from scipy.spatial.distance import cosine

# # Calculate the cosine similarity of the two embeddings.
# sim = 1 - cosine(sen_emb, word_emb)

# print('Cosine similarity: {:.2}'.format(sim))


In [ ]:

# C0085207
# Gestational Diabetes
# C0011849
# Diabetes Mellitus
# C0003811
# Cardiac Arrhythmia
# C0007222
# Cardiovascular Diseases

In [19]:
# text1 = 'C0003811 Cardiac Arrhythmia'
# word1 = 'C0003811'
# text2 = 'C0007222 Cardiovascular Diseases'
# word2 = 'C0007222'
# text1 = 'Gestational Diabetes'
# word1 = 'Diabetes'
# text2 = 'Diabetes Mellitus'
# word2 = 'Mellitus'
text1 = 'Cholera due to vibrio cholerae'
word1 = 'Cholera'
# text1 = 'Cardiac Arrhythmia'
# word1 = 'Cardiac'
# text2 = 'Cardiovascular Diseases'  # Diabetes Mellitus
# word2 = 'Cardiovascular'
text2 = 'Cholera due to vibrio cholerae el tor'
word2 = 'vibrio'

tokenized_text = tokenizer.tokenize(text1)
print(tokenized_text)
tokenized_text = tokenizer.tokenize(text2)
print(tokenized_text)

sen_emb1, word_emb1 = get_embedding(model, tokenizer, text1, word1)
sen_emb2, word_emb2 = get_embedding(model, tokenizer, text2, word2)

from scipy.spatial.distance import cosine

print(word_emb1[:5])
print(word_emb2[:5])

# Calculate the cosine similarity of the two embeddings.
sim = 1.0 - cosine(word_emb1, word_emb2)
# print(sim)
print('word Cosine similarity: {:.4}'.format(sim))

sim = 1.0 - cosine(sen_emb1, sen_emb2)
# print(sim)
print('sent Cosine similarity: {:.4}'.format(sim))

print(sen_emb1[:5])
print(sen_emb2[:5])

['cholera', 'due', 'to', 'vibrio', 'cholerae']
['cholera', 'due', 'to', 'vibrio', 'cholerae', 'el', 'tor']
[-0.0585447  -0.38258275 -0.15768597 -0.06286092 -0.39517486]
[ 0.24277705  0.05865506 -0.03173161  0.02194069 -0.41628045]
word Cosine similarity: 0.8596
sent Cosine similarity: 0.9817
[-0.09673499 -0.01326337 -0.15043162  0.11172631 -0.2900265 ]
[-0.10404948  0.01963876 -0.10541455  0.03605231 -0.34910363]
